In [4]:

!apt-get -qq update
!apt-get -qq install mpich
!pip install mpi4py


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libslurm37.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libslurm37_21.08.5-2ubuntu1_amd64.deb ...
Unpacking libslurm37 (21.08.5-2ubuntu1) ...
Selecting previously unselected package hwloc-nox.
Preparing to unpack .../hwloc-nox_2.7.0-2ubuntu1_amd64.deb ...
Unpacking hwloc-nox (2.7.0-2ubuntu1) ...
Selecting previously unselected package libmpich12:amd64.
Preparing to unpack .../libmpich12_4.0-3_amd64.deb ...
Unpacking libmpich12:amd64 (4.0-3) ...
Selecting previously unselected package mpich.
Preparing to unpack .../archives/mpich_4.0-3_amd64.deb ...
Unpacking mpich (4.0-3) ...
Selecting previously unselected package libmpich-dev:amd64.
Preparing to unpack .../libmpich-dev_4.0-3_amd64.deb ...
Unpacking libmpich

In [4]:
%%writefile mpi_vector_sum_bonus.c
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

// Task: Distributed Partial Summation + Average using MPI_Allreduce

int main(int argc, char* argv[]) {
    int rank, size;
    long N = 10000000;               // Vector size
    double *A = NULL;                // Full vector (root only)
    double *local_A = NULL;          // Local part for each process
    double local_sum = 0.0, global_sum = 0.0;
    double start_time, end_time, parallel_time, serial_time;

    //  Initialize MPI environment
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Task 1: Vector Initialization (root only)
    long base = N / size;
    long remainder = N % size;
    long local_n = base + (rank < remainder ? 1 : 0);

    int *counts = (int*)malloc(size * sizeof(int));
    int *displs = (int*)malloc(size * sizeof(int));

    displs[0] = 0;
    for (int i = 0; i < size; i++) {
        counts[i] = (int)(base + (i < remainder ? 1 : 0));
        if (i > 0) displs[i] = displs[i - 1] + counts[i - 1];
    }

    if (rank == 0) {
        A = (double*)malloc(N * sizeof(double));
        for (long i = 0; i < N; i++)
            A[i] = i + 1; // A[i] = i + 1
    }

    local_A = (double*)malloc(local_n * sizeof(double));

    //  Task 2: Data Distribution (uneven partitions using MPI_Scatterv)
    MPI_Scatterv(A, counts, displs, MPI_DOUBLE, local_A, local_n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    //  Task 3: Local Computation (partial sum)
    start_time = MPI_Wtime();
    for (long i = 0; i < local_n; i++)
        local_sum += local_A[i];

    //  Task 4: Reduction + Average using MPI_Allreduce
    MPI_Allreduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
    double global_avg = global_sum / N;
    end_time = MPI_Wtime();
    parallel_time = end_time - start_time;

    // Task 5: Serial Computation
    if (rank == 0) {
        double serial_sum = 0.0;
        double serial_start = MPI_Wtime();
        for (long i = 0; i < N; i++)
            serial_sum += A[i];
        double serial_end = MPI_Wtime();
        serial_time = serial_end - serial_start;

        //  Display Results
        double expected = (N * (N + 1)) / 2.0;
        printf("\n=== MPI Parallel Vector Sum + Average ===\n");
        printf("Processes        : %d\n", size);
        printf("Total Elements   : %ld\n", N);
        printf("Computed Sum     : %.0f\n", global_sum);
        printf("Expected Sum     : %.0f\n", expected);
        printf("Difference       : %.5f\n", expected - global_sum);
        printf("Computed Average : %.5f\n", global_avg);
        printf("Parallel Time    : %.6f sec\n", parallel_time);
        printf("Serial Time      : %.6f sec\n", serial_time);
        printf("Speedup          : %.2fx\n", serial_time / parallel_time);
    }

    //  Cleanup
    free(local_A);
    free(counts);
    free(displs);
    if (rank == 0) free(A);

    MPI_Finalize();
    return 0;
}


Writing mpi_vector_sum_bonus.c


In [5]:
!mpicc mpi_vector_sum_bonus.c -o mpi_vector_sum_bonus


In [6]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_vector_sum_bonus



=== MPI Parallel Vector Sum + Average ===
Processes        : 4
Total Elements   : 10000000
Computed Sum     : 50000005000000
Expected Sum     : 50000005000000
Difference       : 0.00000
Computed Average : 5000000.50000
Parallel Time    : 0.011379 sec
Serial Time      : 0.035920 sec
Speedup          : 3.16x
